# Exercise Questions

This version of the exercises uses the [WHO Life Expectancy](https://www.kaggle.com/kumarajarshi/life-expectancy-who) dataset includes observations of various health indicators in 193 countries from 2000 to 2015. It's commonly used in regression model tutorials as it has several strong predictors with linear relationships to life expectancy in each country. We will use it here as a simple modeling example that will allow us to focus on model metrics and evaluation procedures.

The dataset includes the following information:
- country, year, and status as a developed or developing nation
- life expectancy, including adult and infant mortality
- public health indicators such as alcohol consumption, average BMI
- income and wealth indicators
- healthcare expenditure (total and as % of GDP)
- immunization rates for hepatitis B, polio, measles, etc.

In [ ]:
# Import packages and dataset
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv("life_expectancy.csv")
df["status"] = [1 if d == "Developed" else 0 for d in df["status"]]
df = df.fillna(0)
df = df.reset_index()

## Explore the Dataset

Take a few minutes to explore the dataset. How many potential predictors are there for a model predicting `life_expectancy`?

In [ ]:
df.info()
#df.head()

In [ ]:
# Create a correlation matrix to identify potential predictors
df_corr = df.corr().round(2)

# Setting a threshold for correlation values; optional
y_corr = df_corr["life_expectancy"][np.absolute(df_corr["life_expectancy"])>0.3]
y_corr

In [ ]:
# Explore the shape of relationships 
sns.pairplot(df[y_corr.index]);

## Set up for model

1. Compare at least two models using scikit-learn. You can view the list of available linear models with scikit-learn [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model). Use at least 2 metrics from scikit-learn's regression evaluation metrics [here](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# Set up X to only include selected column names
X = df[y_corr.index]

# Fill missing data with 0; you can change this if you choose
X = X.fillna(0)

# Drop outcome variable
X = X.drop("life_expectancy", axis = "columns")

# Get y
y = df["life_expectancy"]

## Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

## Standardize the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Fit the Model

Fit the first model and evaluate the model metrics. Compare it to at least one other model and select one going forward.

In [ ]:
lr = LinearRegression()

# Fit the model
lr.fit(X_train, y_train)

# Generate predictions based on the test set 
y_preds = lr.predict(X_test)

In [ ]:
print(f"{lr} training set r2 score: {lr.score(X_train, y_train).round(2)}")
print(f"{lr} test set r2 score: {r2_score(y_test, y_preds).round(2)}")
# Choose another metric

In [ ]:
# Compare to another model

## Modeling for Explanation

Modeling for explanation generally requires more detailed information on individual predictors than scikit-learn provides. In order to model for explanation, we will leverage the `statsmodels` library to explore our model further.

In [ ]:
import statsmodels.api as sm

In [ ]:
x_train = sm.add_constant(X_train)
model = sm.OLS(y_train, x_train)
results = model.fit()

In [ ]:
print(results.summary(xname = ["const"] + list(X.columns)))

2. Explore the `statsmodels` OLS regression summary above. What features are _not_ statistically significant predictors of `life_expectancy` in this model? Try removing them (drop them and recreate the training/test set) and re-evaluate the strength and directionality of the t-values. 

3. The WHO is seeking to identify two or three highest value health targeting programs they can use to improve life expectancy in several countries. Generate (1) a summary of results, (2) a series of visualizations of key predictors, and (3) an overall recommendations on what health issues to pursue a program for.